In [ ]:
import pandas as pd

In [ ]:
# сейчас я буду анализировать значения по столбцам, рассчитывать базовые статистики, находить выбросы
hourses = pd.read_csv('https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/statistics_basics/horse_data.csv', 'r', delimiter = ',', 
                      header=None, na_values = '?')

hourses.rename(columns = {hourses.columns[0]: "surgery?", hourses.columns[1]: 'Age', hourses.columns[3]: 'rectal temperature', 
                          hourses.columns[4]: 'pulse' , hourses.columns[5]: 'respiratory rate', hourses.columns[6]: 'temperature of extremities',
                          hourses.columns[10]: 'pain', hourses.columns[22]: 'outcome'}, inplace = True)

hourses = hourses.drop(hourses.columns[[2, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27]], axis = 1)

hourses

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,9,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...
295,1.0,1,NaN,120.0,70.0,4.0,2.0,3.0
296,2.0,1,37.2,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.5,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.5,100.0,24.0,3.0,3.0,1.0


In [ ]:
# выбросы для каждого значения
q1_rt = hourses['rectal temperature'].quantile(0.25)
q3_rt = hourses['rectal temperature'].quantile(0.75)
iqr_rt = q3_rt - q1_rt
lower_bound_rt = q1_rt - (1.5 * iqr_rt)
upper_bound_rt = q3_rt + (1.5 * iqr)
hourses['rectal temperature'][~hourses['rectal temperature'].between(lower_bound_rt, upper_bound_rt, inclusive = True)].dropna(how = 'all')


44     35.4
80     36.4
118    36.5
141    36.0
238    36.1
251    36.6
298    36.5
Name: rectal temperature, dtype: float64

In [ ]:
q1_p = hourses['pulse'].quantile(0.25)
q3_p = hourses['pulse'].quantile(0.75)
iqr_p = q3_p - q1_p
lower_bound_p = q1_p - (1.5 * iqr_p)
upper_bound_p = q3_p + (1.5 * iqr_p)
hourses['pulse'][~hourses['pulse'].between(lower_bound_p, upper_bound_p, inclusive = True)].dropna(how = 'all')


3      164.0
41     150.0
55     160.0
255    184.0
275    150.0
Name: pulse, dtype: float64

In [ ]:
q1_rr = hourses['respiratory rate'].quantile(0.25)
q3_rr = hourses['respiratory rate'].quantile(0.75)
iqr_rr = q3_rr - q1_rr
lower_bound_rr = q1_rr - (1.5 * iqr_rr)
upper_bound_rr = q3_rr + (1.5 * iqr_rr)
hourses['respiratory rate'][hourses['respiratory rate'].between(lower_bound_rr, upper_bound_rr, inclusive = True)].dropna(how = 'all')


0      28.0
1      20.0
2      24.0
4      35.0
6      16.0
       ... 
294    16.0
296    24.0
297    30.0
298    24.0
299    20.0
Name: respiratory rate, Length: 225, dtype: float64

In [ ]:
hourses[['rectal temperature', 'pulse', 'respiratory rate']].describe()

,rectal temperature,pulse,respiratory rate
count,240.000000,276.000000,242.000000
mean,38.167917,71.913043,30.417355
std,0.732289,28.630557,17.642231
min,35.400000,30.000000,8.000000
25%,37.800000,48.000000,18.500000
50%,38.200000,64.000000,24.500000
75%,38.500000,88.000000,36.000000
max,40.800000,184.000000,96.000000


In [ ]:
# размах
hourses[['rectal temperature', 'pulse', 'respiratory rate']].max() - hourses[['rectal temperature', 'pulse', 'respiratory rate']].min()

rectal temperature      5.4
pulse                 154.0
respiratory rate       88.0
dtype: float64

In [ ]:
# медиана
hourses[['rectal temperature', 'pulse', 'respiratory rate']].median()

rectal temperature    38.2
pulse                 64.0
respiratory rate      24.5
dtype: float64

In [ ]:
# мода
hourses[['rectal temperature', 'pulse', 'respiratory rate']].mode()

,rectal temperature,pulse,respiratory rate
0,38.0,48.0,20.0


In [ ]:
# расчёт количества пропусков, принятие решения о методах их заполнения, создание новго датафрейма с уже заполненными пропусками
print(hourses.isna().sum())

surgery?                       1
Age                            0
rectal temperature            60
pulse                         24
respiratory rate              58
temperature of extremities    56
pain                          55
outcome                        1
dtype: int64


In [ ]:
hourses_full = hourses.copy()

In [ ]:
# была выбрана мода, потому что значения в столбцах означают определённые категории, далее - то же самое. группирую по фактору возраста, тк он есть во всех строках и может влиять
hourses_full['surgery?'] = hourses_full['surgery?'].fillna(hourses_full.groupby('Age')['surgery?'].transform(lambda x: x.mode()[0]))

In [ ]:
hourses_full['temperature of extremities'] = hourses_full['temperature of extremities'].fillna(hourses_full.groupby('Age')['temperature of extremities'].transform(lambda x: x.mode()[0]))

In [ ]:
hourses_full['pain'] = hourses_full['pain'].fillna(hourses_full.groupby('Age')['pain'].transform(lambda x: x.mode()[0]))

In [ ]:
hourses_full['outcome'] = hourses_full['outcome'].fillna(hourses_full.groupby('Age')['outcome'].transform(lambda x: x.mode()[0]))

In [ ]:
# эти значения напрямую являются мерами измерений, медиана более стабильна, беру её для последующих, так же группирую по возрасту
hourses_full['rectal temperature'] = hourses_full['rectal temperature'].fillna(hourses_full.groupby('Age')['rectal temperature'].transform(lambda x: x.median()))

In [ ]:
hourses_full['pulse'] = hourses_full['pulse'].fillna(hourses_full.groupby('Age')['pulse'].transform(lambda x: x.median()))

In [ ]:
hourses_full['respiratory rate'] = hourses_full['respiratory rate'].fillna(hourses_full.groupby('Age')['respiratory rate'].transform(lambda x: x.median()))

In [ ]:
# проверка
print(hourses_full.isna().sum())

surgery?                      0
Age                           0
rectal temperature            0
pulse                         0
respiratory rate              0
temperature of extremities    0
pain                          0
outcome                       0
dtype: int64
